# language tools

Converting between languages, parsing languages computer languages, demonstrating tools like syntax hilighters, and outputting the diffed code


## introduction



TODO: see if anything can be learned from https://www.rascal-mpl.org/


TODO: use syntax tree to convert from and to any language


use a common library for language generation 

https://bitbucket.org/birkenfeld/pygments-main/src/default/pygments/lexers/haskell.py

https://github.com/PrismJS/prism/blob/master/components/prism-markdown.js

http://qbnz.com/highlighter/

https://stackoverflow.com/questions/171924/how-do-i-syntax-check-a-bash-script-without-running-it

bash -n scriptname

Improve on the babel-preset-php pattern of replacing code with a tree, just use templates and the tree creator on regular code blocks

https://gitlab.com/kornelski/babel-preset-php/blob/master/lib/trans.js

```
function translateAssignBlock(node, scope) {
    if (isVariableCreatingAssignment(node, scope)) {
        return makeVariableDeclaration(node.left.name, "var", translateExpr(node.right, scope), node.loc, scope);
    }
    return wrapInStatement(translateAssign(node, scope));
}
```

should be something like:
```
var ASSIGMENT_EXPRESSION = (leftName, right) => {
    var leftName = right;
}

function translateAssignBlock(node, scope) {
    if(//Assignment[/Identifier].includes(node)) {
        return getAST(ASSIGMENT_EXPRESSION, node)
    }
    return getAST(STATEMENT, node)
}
```

that way when they are written in multiple languages we can transpile any language, maybe

use this library as a reference for lots of languages https://github.com/seaneshbaugh/rosetta-euler

automatically translate https://projecteuler.net/archives to any language

https://wiki.haskell.org/Euler_problems


TODO: use babel and a transpiling kernel from babel-preset-php, 

TODO: make a generic translator that does manipulations on queries based on templates






In [ ]:
// sample definition for MatLab



In [ ]:
// definition for nodejs



## select multilungual tree



### TODO: select jison tree

Because parsing esprima code isn't enough! Let's use syntax tree features in any language. The Jison parse comes from Bison, which has lots of support

Tree-sitter also looks nice https://github.com/tree-sitter/tree-sitter-c

https://github.com/zaach/jison

https://en.m.wikipedia.org/wiki/GNU_Bison

Works just like selectAst() except no esprima, generates the tree using jison instead, so it requires a language specifier either in a comment at the top of the code or in a parameter.  It can also read a file if it exists, so one more check in addition to reading a string, and the language can be derived from the file extension.



#### the code

select jison tree?

TODO: unfail this idea if `.y` files are found for the languages I want to use.



In [ ]:
var fs = require('fs');
var jison = require('jison');
var importer = require('../Core')
var {
    selectDom,
    evaluateDom,
    treeToHtml,
    htmlToTree
} = importer.import(['select tree', 'tree to html', 'html to tree'])

function getParser(jisonFile) {
    var bnf
    if(fs.existsSync(jisonFile)) {
        bnf = fs.readFileSync(jisonFile, 'utf8')
    } else {
        bnf = importer.interpret(jisonFile).code
    }
    
    var parser = new jison.Parser(bnf)
    return parser
}


function toString(ctx, subctx) {
    var ast = (subctx || ctx)
    if(typeof ast.ownerDocument !== 'undefined') {
        ast = htmlToTree(ast)
    }
    //escodegen.attachComments(ast, ast.comments, ast.tokens, ast.whitespace)
    return escodegen.generate(ast, {
        comment: true,
        tokens: true,
        whitespace: true,
        format: {indent: {style: '    '}, quotes: 'single'}
    })
}

function selectCode(descriptor, code, language) {
    if(typeof code === 'function') {
        code = code.toString()
    }
    if(typeof code === 'string') {
        if(fs.existsSync(code)) {
            code = fs.readFileSync(code)
            if(typeof language === 'undefined') {
                language = 
            }
        }
        var parser = getParser(language)
        try {
            code = esprima.parse(code, {range: true, tokens: true,
                                        comment: true, whitespace: true})
        } catch(e) {
            console.log(e)
            throw new Error('Could not parse \n' + code)
        }
    }
    // TODO: move this conversion to selectDom for all objects?
    //   No more selectTree because jsel and friends are inconsistent.
    if(typeof code.evaluate === 'undefined'
       && typeof code.ownerDocument === 'undefined') {
        code = selectDom('//Program', treeToHtml(code))
        code.ownerDocument.toString = toString.bind(code, code)
    }
    return selectDom(descriptor, code)
}

module.exports = {
    selectCode
}


### select antlr tree

Using antlr templates because look at all the glorious grammars posted right there on it's main Github page! https://github.com/antlr/grammars-v4

select antlr tree?


In [ ]:
var fs = require('fs');
var {CommonTokenStream, InputStream} = require('antlr4');
var importer = require('../Core')
var {
    selectDom,
    evaluateDom,
    treeToHtml,
    htmlToTree
} = importer.import(['select tree', 'tree to html', 'html to tree'])

function getParser(language) {
    // TODO: get parser by language lookup
    const {CLexer} = require('../Resources/Parsers/grammars-v4-master/c/CLexer.js')
    const {CParser} = require('../Resources/Parsers/grammars-v4-master/c/CParser.js')

    return (input) => {
        //const parser = new GenericParser()
        const lexer = new CLexer(new InputStream(input))
        const parser = new CParser(new CommonTokenStream(lexer))
        parser.buildParseTrees = true;
        return parser
    }
}

//TODO: fix this with own transpiler because no one is going to share their hard work on StringTemplate4 code
function toString(ctx, subctx) {
    var ast = (subctx || ctx)
    if(typeof ast.ownerDocument !== 'undefined') {
        ast = htmlToTree(ast)
    }
    //escodegen.attachComments(ast, ast.comments, ast.tokens, ast.whitespace)
    return escodegen.generate(ast, {
        comment: true,
        tokens: true,
        whitespace: true,
        format: {indent: {style: '    '}, quotes: 'single'}
    })
}

function selectCode(descriptor, code, language) {
    if(typeof code === 'function') {
        code = code.toString()
    }
    if(typeof code === 'string') {
        if(fs.existsSync(code)) {
            code = fs.readFileSync(code).toString('utf8')
            if(typeof language === 'undefined') {
                // TODO: use language file extension from media-server code module gheshi support
            //    language = 
            }
        }
        var parser = getParser(language)
        try {
            code = parser(code)
            return code
        } catch(e) {
            console.log(e)
            throw new Error('Could not parse \n' + code)
        }
    }
    // No more selectTree because jsel and friends are inconsistent/partially implemented.
    if(typeof code.evaluate === 'undefined'
       && typeof code.ownerDocument === 'undefined') {
        code = selectDom('//Program', treeToHtml(code))
        code.ownerDocument.toString = toString.bind(code, code)
    }
    return selectDom(descriptor, code)
}

module.exports = {
    selectCode
}


#### TODO: test select jison on some quake 3 C code



In [8]:
var antlr = require('antlr4')
var importer = require('../Core')
var {selectCode} = importer.import('select antlr tree')

var tree = selectCode('*', '/Users/briancullinan/planet_quake/code/q3_ui/ui_cdkey.c')
console.log(tree.compilationUnit())

CompilationUnitContext {
  parentCtx: null,
  invokingState: -1,
  ruleIndex: 82,
  children: [
    TerminalNodeImpl {
      parentCtx: [Circular],
      symbol: [CommonToken],
      invokingState: 1263
    }
  ],
  start: CommonToken {
    source: [ [CLexer], [InputStream] ],
    type: -1,
    channel: 0,
    start: 7273,
    stop: 7272,
    tokenIndex: 0,
    line: 292,
    column: 0,
    _text: null
  },
  stop: null,
  exception: null,
  parser: CParser {
    _listeners: [ ConsoleErrorListener {} ],
    _interp: ParserATNSimulator {
      atn: [ATN],
      sharedContextCache: [PredictionContextCache],
      parser: [Circular],
      decisionToDFA: [Array],
      predictionMode: 1,
      _input: null,
      _startIndex: 0,
      _outerContext: null,
      _dfa: null,
      mergeCache: null
    },
    _stateNumber: -1,
    _input: CommonTokenStream {
      tokenSource: [CLexer],
      tokens: [Array],
      index: 0,
      fetchedEOF: true,
      channel: 0
    },
    _errHandler: De

#### TODO: transpile to languages and frameworks


In [ ]:
// Each composite transpiler instance has the ability to traverse the parse tree
// for a specific language with its 'visit*' methods, and generate output code for 
// another language with its 'emit*' methods.
const getJavascriptVisitor = require('./codegeneration/javascript/Visitor');
const getJavaGenerator = require('./codegeneration/java/Generator');
const getPythonGenerator = require('./codegeneration/python/Generator');
...

const loadJSTree = (input) => {
/* Lexing and parsing the user input */
  ...
};

/**
 * Compose a transpiler and return a compile method that will use that transpiler
 * to visit the tree and return generated code.
 *
 * @param {function} loadTree - the method takes in user input and returns a tree.
 * @param {Visitor} visitor - the input-language visitor.
 * @param {function} generator - returns a generator that inherits from it’s arg.
 * 
 * @returns {function} the compile function to be exported
 */
const composeTranspiler = (loadTree, visitor, generator) => {
  const Transpiler = generator(visitor);
  const transpiler = new Transpiler();

  return {
    compile: (input) => {
      const tree = loadTree(input);
      return transpiler.start(tree);
    }
  };
}

module.exports = {
  javascript: {
    java: composeTranspiler(
      loadJSTree,
      getJavascriptVisitor(JavascriptANTLRVisitor), // Visitor + ANTLR visitor
      getJavaGenerator // Method that takes in a superclass, i.e. the visitor
    ),
    python: composeTranspiler(
      loadJSTree,
      getJavascriptVisitor(JavascriptANTLRVisitor)),
      getPythonGenerator
    ),
    ...
   },
   ...
}

## transpiling tools



### transpile code

Pretty nice syntax but does it work on non, C-style languages.

This is probably the closest example I could find in recent research:

https://blog.scottlogic.com/2016/06/22/xslt-inspired-ast-transforms.html

With one small twist. Matches are derived automatically from code in the current language.


#### the code

transpile code?

In [ ]:
var importer = require('../Core')
var {htmlToTree} = importer.import('html to tree')
var {selectAst} = importer.import('select code tree')

function add() {
}

function replace() {
}

function remove(ctx) {
    if(!Array.isArray(ctx)) {
        ctx = [ctx]
    }
    ctx.forEach(o => {
        var body = selectAst([`./parent::*`], o)[0]
        body.removeChild(o)
    })
}

function transpile(input, output, code) {
    var root
    if(Array.isArray(input) && typeof code === 'undefined') {
        code = output
    }
    root = selectAst(`//Program`, code)
    input.forEach(i => {
        selectAst([...i], root)
    })
    // TODO: add a feature for parsing default parameters
    // TODO: check if tree string and tree dom and tree ast are changed
    /*
    if()
    } else {
        root = selectAst(`//Program`, code)
        selectAst([input, output], root)
    }
    */
    return root
}

module.exports = {
    transpile,
    remove,
    replace
}


#### test transpiling with core module

In [ ]:
var importer = require('../Core')
var {replaceCore} = importer.import('replace core requirement')

var code = `
var importer = require('../Core');
var getArrayAST = importer.import('get ast path array');
`

if(typeof $$ != 'undefined') {
    $$.mime({'text/plain': replaceCore(code)})
    
    /*
    expected output 
var getArrayAST = importer.import('get ast path array');
*/
    
}



#### TODO: test using CSS style


In [ ]:


//""`*[type*='StaticMemberExpression'] > *[name=="importer"]`;
//var $ = cheerio.load(page);


#### TODO: convert between astq and other tree libraries

How does this work

https://github.com/coderaiser/estree-to-babel



In [ ]:

var keywords = ['type', 'id', 'params',
                'body', 'generator', 'async',
                'range', 'expression', 'callee',
                'arguments'];


#### TODO: convert trees using this library



In [ ]:
const fs = require('mz/fs');
const xpath = require('xpath');
const parse5 = require('parse5');
const xmlser = require('xmlserializer');
const dom = require('xmldom').DOMParser;

(async () => {
    const html = await fs.readFile('./test.htm');
    const document = parse5.parse(html.toString());
    const xhtml = xmlser.serializeToString(document);
    const doc = new dom().parseFromString(xhtml);
    const select = xpath.useNamespaces({"x": "http://www.w3.org/1999/xhtml"});
    const nodes = select("//x:a/@href", doc);
    console.log(nodes);
})();



### babel transpiler plugin

Add a plugin for babel to the babel jupyter notebook can be used.


#### the code

babel transpiler?


In [ ]:
var babel = require('babel-core')
var toBabel = require('estree-to-babel');
var importer = require('../Core')
var {selectAst} = importer.import('select code tree')
var {selectAcorn} = importer.import('select acorn tree')
var {htmlToTree} = importer.import('html to tree')
var {addImports} = importer.import('add missing imports')
var {replaceCore} = importer.import('replace core requirement')
var {replaceImports} = importer.import('replace notebook import')

// TODO: convert these to transpiler tools like Add, Replace, Remove
var transpilePlugins = [
    addImports,
    replaceImports,
    replaceCore,
]

function transpileNotebook() {
    return {
        manipulateOptions(opts, parserOpts) {
            opts.parserOpts = {
                parser(code, opts) {
                    code = transpilePlugins.reduce((code, plugin) => {
                        return plugin(code, opts.sourceFileName)
                    }, code)
                    var ast = toBabel(code)
                    return ast
                },
            };
        },
    };
}

function babelTranspile() {
    return {
        plugins: [
            transpileNotebook
        ]
    }
}

module.exports = babelTranspile



#### example and test of babel transpiler

This would be copied in to babel-jupyter-preset.js and included in .babelrc and used with js-babel jupyter kernel.

https://github.com/n-riesco/jp-babel



In [ ]:
var babel = require('babel-core')
var importer = require('../Core')
var babelTranspile = importer.import('babel transpiler')

var out = babel.transform(`
// this code is missing an import
addImports('// some test code')
`, {
    presets: [babelTranspile],
    plugins: [/* 'estree' */],
    filename: 'test_code.js'
}).code;

console.log(out)

/* expected output

var addImports = require('./')

*/


### TODO: Find all TODO items in code and in JSON notebooks




### TODO: show the power by making a calculator in fewer lines of code

### TODO: convert between languages

Improve on the babel-preset-php pattern of replacing code with a tree, just use templates and the tree creator on regular code blocks




#### convert php with less code


https://gitlab.com/kornelski/babel-preset-php/blob/master/lib/trans.js


In [ ]:

// make sure this format works both directions
// from PHP to javascript
var fromPhpToJavascropt = {
    '//CallExpression[@name="array_shift"]': {
        
    }
}

var fromJavascriptToPhp = {
    
}

var phpToJs = [
    [() => array_shift(arr), () => arr.shift()],
    [() => strpos(str, match), () => { // has parameters so call this function instead?
        var i = str.indexOf(match);
        return i > -1 ? i : false}],
    [() => substr(str, start, length), () => str.substr(start, length)],
    [() => is_numeric(num), () => !isNaN(parseInt(num)) || !isNaN(parseFloat(num))],
    [() => abs(), () => Math.abs()], // doesn't mess with parameters
    // Call Identifier name match done automatically
    //[['abs', 'sin', 'cos', 'pow', 'floor', 'ceil', 'round', 'max', 'min'], {
    //    `//*[contains(@type, "Call")`: (id) => Math.id(), // optional ()
    //}]
    // above is shorthand for this:
    [(id = ['abs', 'sin', 'cos',
            'pow', 'floor', 'ceil',
            'round', 'max', 'min']) => id(), {
        `//*[contains(@type, "Call")`: (id) => Math.id(), // optional ()
    }]

]

// this will work as long as PHP syntax is also called a Call or Member or Indentifier
var jsToPhp = phpToJs.map(d => [d[1], d[0]])


#### convert this jupyter kernel to matlab

https://github.com/WolframResearch/WolframLanguageForJupyter/blob/master/WolframLanguageForJupyter/Resources/SocketUtilities.wl



```
	Needs["ZeroMQLink`"]; (* socketWriteFunction, ZeroMQLink`Private`ZMQWriteInternal,
								ZeroMQLink`ZMQSocketWriteMessage *)

(************************************
	private symbols
*************************************)

	(* begin the private context for WolframLanguageForJupyter *)
	Begin["`Private`"];

(************************************
	utility for writing a part
		of a message frame to a
		socket
*************************************)

	(* write a part of a message frame to a socket *)
	(* adjust for differences in Wolfram Engine version *)
	If[TrueQ[$VersionNumber < 12.0],
		Options[socketWriteFunction] = {"Asynchronous"->False,"Multipart"->False};
		socketWriteFunction[sock_, data_List, opts:OptionsPattern[]] := ZeroMQLink`Private`ZMQWriteInternal[sock, data, opts];
		socketWriteFunction[sock_, data_ByteArray, rest___]:= socketWriteFunction[sock, Normal[data], rest]
		,
		socketWriteFunction = ZeroMQLink`ZMQSocketWriteMessage
	];

(************************************
	utility for writing a message
		frame to a socket
*************************************)

	(* write a message frame that matches Jupyter's messaging protocols to a socket *)
	sendFrame[socket_, frame_Association] := Module[{},
		
		(* see https://jupyter-client.readthedocs.io/en/stable/messaging.html for an explanation of the below *)
		
		socketWriteFunction[
			socket, 
			frame["ident"],
			"Multipart" -> True
		];

		socketWriteFunction[
			socket, 
			StringToByteArray[#1],
			"Multipart" -> True
		]& /@ Lookup[frame, {"idsmsg", "signature", "header", "pheader", "metadata"}];

		socketWriteFunction[
			socket, 
			StringToByteArray[frame["content"]],
			"Multipart" -> False
		];
	];
    
    ```

#### translation formats test

Should only need one description format from notebook module exporter to improve

jison and bison specifications could be imported and used automatically

https://www.npmjs.com/package/jison

https://www.gnu.org/software/bison/

Then I won't need jsdom, or any custom language stuff


In [ ]:

// format I used on study sauce templates

var array_shift = function (arr) {return arr.shift();};
var strpos = function (str, match) {var i; return (i = str.indexOf(match)) > -1 ? i : false;};
var substr = function (str, start, length) {return str.substr(start, length);};
var is_numeric = function (num) {return !isNaN(parseInt(num)) || !isNaN(parseFloat(num));};
var strlen = function (str) {return (''+(str || '')).length;};
var array_merge = function () {
var isObject = typeof arguments[0] == 'object' && arguments[0] != null && arguments[0].constructor != Array;
var args = [isObject ? {} : []];
for(var a = 0; a < arguments.length; a++) {
    args[args.length] = arguments[a];
};
return args.reduce(function (a, b) {
    return isObject ? $.extend(a, b) : $.merge(a, b);
});
};
var round = function (num, digits) {
    if(digits > 0) {
        return Math.round(num * (10 * (digits ? digits : 0))) / (10 * (digits ? digits : 0));
    }
    return Math.round(num);
};
var is_string = function (obj) {return typeof obj == 'string';};
var is_a = function (obj, typeStr) { return typeof obj == 'object' && obj != null && obj.constructor.name == typeStr();};
var intval = function (str) {var result = parseInt(str); return isNaN(result) ? 0 : result;};
var trim = function (str) {return (str || '').trim();};
var explode = function (del, str) {return (str || '').split(del);};
var array_slice = function (arr, start, length) { return (arr || []).slice(start, length); };
var array_splice = function (arr, start, length) { return arr.splice(start, length); };
var array_search = function (item, arr) { var index = (arr || []).indexOf(item); return index == -1 ? false : index; };
var count = function (arr) { return (arr || []).length; };
var in_array = function (needle, arr) { return (arr || []).indexOf(needle) > -1; };
var array_values = function (arr) { return (arr || []).slice(0); };
var is_array = function (obj) { return typeof obj == 'array' || typeof obj == 'object'; }; // PHP and javascript don't make a distinction between arrays and objects syntax wise using [property], all php objects should be restful anyways
var array_keys = function (obj) {var result=[]; for (var k in obj) { if (obj.hasOwnProperty(k)) { result[result.length] = k } } return result; };
var implode = function (sep, arr) {return (arr || []).join(sep);};
var preg_replace = function (needle, replacement, subject) {
    return (subject || '').replace(new RegExp(needle.split('/').slice(1, -1).join('/'), needle.split('/').slice(-1)[0] + 'g'), replacement);
};
var number_format = function (num, digits) { return parseFloat(num).toFixed(digits);};
var preg_match = function (needle, subject, matches) {
    var result = (new RegExp(needle.split('/').slice(1, -1).join('/'), needle.split('/').slice(-1)[0] + 'g')).exec(subject);
    if(result == null) {
        return 0;
    }
    if(typeof matches != 'undefined') {
        for(var m = 0; m < result.length; m++) {
            matches[m] = result[m];
        }
    }
    return result.length;
};
var ucfirst = function (str) {return (str || '').substr(0, 1).toLocaleUpperCase() + str.substr(1);};
var str_replace = function (needle, replacement, haystack) {return (haystack || '').replace(new RegExp(RegExp.escape(needle), 'g'), replacement);};
var call_user_func_array = function (context, params) {return context[0][context[1]].apply(context[0], params);};
var print = function (s) { window.views.__output += s };
var strtolower = function(s) { return (s || '').toLocaleLowerCase(); };
var empty = function(s) {
    if(typeof s == 'undefined' || ('' + s).trim() == ''
        || s === false || s === 'false' || s == null || s === 0
        || (typeof s == 'object' && s.constructor == Array && s.length == 0)) {
        return true;
    }
    if(typeof s == 'object') {
        for(var p in s) {
            if(s.hasOwnProperty(p)) {
                return false;
            }
        }
    }
};
var json_encode = JSON.stringify;
var method_exists = function (s,m) { return typeof s == 'object' && typeof s[m] == 'function'; };
var isset = function (s) { return typeof s != 'undefined'; };



### TODO: eval detector

### TODO: translate patterns in to other languages


### TODO: detect weird language situations like typings and errors

TODO: import falsey javascript in to this notebook?


### TODO: automatically console.log values involved in assertions

So annoying to run tests multiple times to debug.

TODO: Depends on working aspects



#### more information

Partly derived from 

https://github.com/gotwarlost/istanbul/blob/master/lib/instrumenter.js




```
this.walker = new Walker({
ArrowFunctionExpression: [ this.arrowBlockConverter ],
ExpressionStatement: this.coverStatement,
ExportNamedDeclaration: this.coverExport,
BreakStatement: this.coverStatement,
ContinueStatement: this.coverStatement,
DebuggerStatement: this.coverStatement,
ReturnStatement: this.coverStatement,
ThrowStatement: this.coverStatement,
TryStatement: [ this.paranoidHandlerCheck, this.coverStatement],
VariableDeclaration: this.coverStatement,
IfStatement: [ this.ifBlockConverter, this.coverStatement, this.ifBranchInjector ],
ForStatement: [ this.skipInit, this.loopBlockConverter, this.coverStatement ],
ForInStatement: [ this.skipLeft, this.loopBlockConverter, this.coverStatement ],
ForOfStatement: [ this.skipLeft, this.loopBlockConverter, this.coverStatement ],
WhileStatement: [ this.loopBlockConverter, this.coverStatement ],
DoWhileStatement: [ this.loopBlockConverter, this.coverStatement ],
SwitchStatement: [ this.coverStatement, this.switchBranchInjector ],
SwitchCase: [ this.switchCaseInjector ],
WithStatement: [ this.withBlockConverter, this.coverStatement ],
FunctionDeclaration: [ this.coverFunction, this.coverStatement ],
FunctionExpression: this.coverFunction,
LabeledStatement: this.coverStatement,
ConditionalExpression: this.conditionalBranchInjector,
LogicalExpression: this.logicalExpressionBranchInjector,
ObjectExpression: this.maybeAddType,
MetaProperty: this.coverMetaProperty,
}, this.extractCurrentHint, this, this.opts.walkDebug);
```


#### the code

Add logging messages to every statement so the logger can suppress the ones we aren't working on.



### TODO: automatically detect where logging is needed



display safe object?

TODO: move this to languages and use with transpiler to create nice logging scripts. Every company I worked for said their logging isn't good enough to be helpful, maybe something can be done automatically in code.


In [ ]:
function toLogString(vals) {
  return vals.map(val => {
    if ('string' === typeof val) return val
    return toSafeString(val)
  }).join(' ') + '\n'
}

function toSafeString(val) {
  let pre = ''
  if ('object' === typeof val && val && val.constructor && val.constructor.name !== 'Object') {
    pre = '[' + val.constructor.name + '] '
  }
  if ('function' === typeof val) {
    return val + ''
  }
  try {
    return pre + JSON.stringify(val)
  } catch (e) {}
  try {
    return pre + '' + val
  } catch (e) {}
  return pre + '[value cannot be rendered]'
}



### TODO: derive imports from usage

The ```require()``` is used and automatically fix ```{expanded}``` or ```assignment =``` imports.

So annoying to get `[This] is not a function`, just because you forgot to ```{expand}``` an import. Why do frameworks even do this, it's stupid.


## TODO: get debugging working in all languages / jupyter

Using transpiling and injection:

https://github.com/gotwarlost/istanbul/blob/master/lib/instrumenter.js

TODO: depends on working aspects.


## TODO: syntax, completion, evaluation, quality tools, similar to the crap that is automated on github.